<a href="https://colab.research.google.com/github/ThesanR/TorontoPermitClassification/blob/main/PermitClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/b6f1907e-5a45-43f7-8c3c-56e4edc4f42f/download/Cleared%20Permits%202016.zip
!unzip -q "Cleared Permits 2016.zip"

Classification of Permit applications from the City of Toronto between the years 2000 and 2023.

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('clearedpermits2016.csv')
df

<ipython-input-5-cb1aac6b2836>:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('clearedpermits2016.csv')


,PERMIT_NUM,REVISION_NUM,PERMIT_TYPE,STRUCTURE_TYPE,WORK,STREET_NUM,STREET_NAME,STREET_TYPE,STREET_DIRECTION,POSTAL,...,DWELLING_UNITS_LOST,EST_CONST_COST,ASSEMBLY,INSTITUTIONAL,RESIDENTIAL,BUSINESS_AND_PERSONAL_SERVICES,MERCANTILE,INDUSTRIAL,INTERIOR_ALTERATIONS,DEMOLITION
0,00 119701 CMB,0,Residential Building Permit,SFD - Detached,Addition to Existing Building,11,BETHNAL,AVE,,M8Y,...,NaN,38000,0.0,0.0,50.0,0.00,0.0,0.0,0.0,0.0
1,00 121417 PLB,0,Plumbing(PS),SFD - Detached,Install/Alter Plumbing - only,142,GOVERNMENT,RD,,M8X,...,NaN,0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0
2,00 123349 CMB,0,Residential Building Permit,SFD - Detached,Addition/Alteration to Existing Building,326,KENNEDY,AVE,,M6P,...,NaN,3000,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0
3,00 123820 CMB,0,Non-Residential Building Permit,Office,Alteration to Existing Building,10,GURNEY,CRES,,M6B,...,NaN,11540,0.0,0.0,0.0,57.78,0.0,0.0,0.0,0.0
4,00 124475 PLB,0,Plumbing(PS),SFD - Detached,Install/Alter Plumbing - only,455,ANNETTE,ST,,M6P,...,NaN,1000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41051,99 251454 BLD,00,Small Residential Projects,SFD - Detached,Addition(s),115,BROOKE,AVE,,M5M,...,NaN,NaN,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
41052,99 251578 BLD,00,Small Residential Projects,SFD - Detached,Addition(s),71,CRAIGMORE,CRES,,M2N,...,NaN,NaN,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
41053,99 251595 BLD,00,Small Residential Projects,SFD - Detached,Addition(s),71,CRAIGMORE,CRES,,M2N,...,NaN,NaN,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
41054,99 251902 PLB,00,Plumbing(PS),SFD - Detached,Building Permit Related(PS),99,ESTELLE,AVE,,M2N,...,NaN,NaN,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
